Import libraries

In [4]:
import os

import pandas as pd

import numpy as np

from scipy import stats
import scipy.stats as sc
from scipy.cluster.hierarchy import linkage, leaves_list

import statsmodels.api as sm

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from learner.mlearner import learn_with_interactions, learn_without_interactions, mean_absolute_percentage_error, sample_random
from learner.model import genModelTermsfromString, Model, genModelfromCoeff

Import data

In [2]:
dataFolderName = '../data_luc/video/'

listVideoName =   ['x264-1908-bridgefar-wasm/x264-results1.csv', 
                   'x264-1908-ice-wasm/x264-results1.csv',
                   'x264-1908-flower-wasm/x264-results1.csv', 
                   'x264-1908-caire-wasm/x264-results1.csv',
                   'x264-0208-sintel-calda-wasm/x264-results1.csv', 
                   'x264-1908-footballcif-wasm/x264-results1.csv',
                   'x264-0308-crowd_run-wasm/x264-results1.csv', 
                   'x264-0608-blue-wasm/x264-results1.csv',
                   'x264-0608-people-wasm/x264-results1.csv', 
                   'x264-1908-sunflowers-wasm/x264-results1.csv',
                   'x264-0408-deadline-wasm/x264-results1.csv', 
                   'x264-2108-bridgeclose-wasm/x264-results1.csv',
                   'x264-1908-husky-wasm/x264-results1.csv', 
                   'x264-1908-tennis-wasm/x264-results1.csv',
                   'x264-1908-riverbed-wasm/x264-results1.csv', 
                   'x264-0608-park-wasm/x264-results1.csv',
                   'x264-0508-soccer-wasm/x264-results1.csv']

# creation of the list of videos (for each video: x264 configurations + measurements)
listVideo = []

for vn in listVideoName:
    listVideo.append(pd.read_csv(open(dataFolderName+vn,"r")))
    
nbVideos = len(listVideo)

# test
print("There are " + str(len(listVideo)) + " videos")
assert (len(listVideoName) == len(listVideo))


#print a csv
listVideo[7].drop(['no_8x8dct', 'no_deblock', 'no_mixed_refs', 'no_weightb', 'size', 'usertime', 'systemtime', 'H264', 'no_asm', 'configurationID'], axis=1)[:10]

There are 17 videos


,no_cabac,no_fast_pskip,no_mbtree,rc_lookahead,ref,elapsedtime
0,False,True,False,20,9,5.074625
1,True,True,False,40,9,5.902625
2,False,False,True,40,1,2.908125
3,True,True,True,40,9,5.027500
4,False,False,False,60,5,4.329125
5,True,True,False,60,5,4.710125
6,False,False,False,60,1,3.624125
7,True,True,False,60,1,3.680875
8,False,True,False,60,1,3.512125
9,False,True,False,60,9,6.309250


# L2s implementation

In [28]:
def evalModel(model, config):
    return float(model.evaluateModelFast(np.matrix(config))[0])
def learnModel(X, y, withInteractions):
    if (withInteractions):
        m = learn_with_interactions(X, y)
    else:
        m = learn_without_interactions(np.asarray(X), np.asarray(y))
    learned_model_terms = genModelfromCoeff(m.named_steps['linear'].coef_, ndim)
    return Model(learned_model_terms, ndim)

# for this tutorial we assume 20 options:
ndim = 20 
# defines the dimension of the model, i.e., the number of variables in the model

In [18]:
# lets see how we evaluate the model with specific input
c0 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c1 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c2 = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c3 = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c23 = [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [19]:
#Let's define a list of configurations and a corresponding list of measurement results
X = [c0, c1, c2, c3, c23]
y = [10, 12, 15, 13, 10]

X, y


([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [10, 12, 15, 13, 10])

In [20]:
learned_model = learnModel(X, y, 0)
learned_model.toString()

'-0.00 * o0 + 1.00 * o1 + -1.00 * o2 + -3.3306690738754696e-16'

In [21]:
learned_model = learnModel(X, y, 1)
learned_model.toString()

'2.00 * o0 + 5.00 * o1 + 3.00 * o2 + -8.00 * o0 * o19 + -8.00 * o1 * o18 + -8.00 * o2 * o17 + -8.00 * o3 * o16 + -8.00 * o4 * o15 + -8.00 * o5 * o14 + -8.00 * o6 * o13 + -8.00 * o7 * o12 + -8.00 * o8 * o11 + -8.00 * o9 * o10 + -8.881784197001252e-16'

In [22]:
# Let's define a model, in this case a polynomial model
# Polynomial models are a great tool for determining which input factors drive responses and in what direction.
# Here we define a model with 20 dimensions, each variable represents a dimension and influence of each variable
# is different, e.g., o0 has the coefficient of 1, while o1 has the coefficient of 2 so its effect is twice comparing
# with o0. Also, we have two terms that represents the interactions of variables, e.g., 3 * o3 * o6.

true_model = """10 + 1.00 * o0 + 2.00 * o1 + 3.00 * o2 +
4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 
1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 
6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 
3.00 * o18 + 4.00 * o19 + 1 * o0 * o1 + 3 * o3 * o6"""


In [23]:
# The model above is just a representation in string, 
# so we need to build a model that we can evaluate given an input
model_terms = genModelTermsfromString(true_model)
true_model = Model(model_terms, ndim)
true_model.toString()

'1.00 * o0 + 2.00 * o1 + 3.00 * o2 + 4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 3.00 * o18 + 4.00 * o19 + 1.00 * o0 * o1 + 3.00 * o3 * o6 + 10.0'

In [24]:
# We could also look up specific measurement results in a secret model
# that we use as ground truth:
# sample_random(ndim=ndim, budget=1000)

y = true_model.evaluateModelFast(np.asarray(X))

X, y


([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 array([10., 11., 12., 13., 15.]))

In [25]:
learned_model = learnModel(X, y, 1)
learned_model.toString()


'1.00 * o0 + 2.00 * o1 + 3.00 * o2 + 0.00 * o0 * o19 + 0.00 * o1 * o18 + 0.00 * o2 * o17 + 0.00 * o3 * o16 + 0.00 * o4 * o15 + 0.00 * o5 * o14 + 0.00 * o6 * o13 + 0.00 * o7 * o12 + 0.00 * o8 * o11 + 0.00 * o9 * o10'

In [26]:
# we first consider sampling some ground truth data, note that we sample again 
# in order to test on data that was not necessarily used for learning the model
X = sample_random(ndim=ndim, budget=10000)
y_true = true_model.evaluateModelFast(X)
y_pred = learned_model.evaluateModelFast(X)
err = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
err



94.05755356456655

nbVideos = len(listVideo)

def mape(y_true, y_pred):
    return np.mean(np.abs((y_true-y_pred)/y_true))

pct_test = 0.3

mat_mape_l2s = np.ones(nbVideos*nbVideos).reshape(nbVideos, nbVideos)

random_state = np.random.randint(0,1000)

for i in range(nbVideos):
    for j in range(nbVideos):

        source = listVideo[i] #i
        X_src = source[['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
            'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']] 
        y_src = np.array(source['elapsedtime'], 
                         dtype=float)

        X_src_train, X_src_test, y_src_train, y_src_test = train_test_split(X_src, 
                                                                            y_src, 
                                                                            test_size=pct_test, 
                                                                            random_state=random_state)

        target = listVideo[j] #j
        X_tgt = target[['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
            'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']] 
        y_tgt = np.array(target['elapsedtime'], 
                         dtype=float)

        X_tgt_train, X_tgt_test, y_tgt_train, y_tgt_test = train_test_split(X_tgt, 
                                                                            y_tgt, 
                                                                            test_size=pct_test, 
                                                                            random_state=random_state)

        coef, intercept, r_value, _ , std_err = stats.linregress(y_src_train, y_tgt_train)

        clf = RandomForestRegressor()
        clf.fit(X_src_train, y_src_train)
        
        y_src_pred_test = clf.predict(X_src_test)
        
        y_tgt_pred_test = intercept + coef*y_src_pred_test

        mat_mape_l2s[i][j] = min(mape(y_tgt_test, y_tgt_pred_test),1)

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.06, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, pd.DataFrame(X[included+[new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = np.argmin(new_pval)
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.5}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, pd.DataFrame(X[included])).fit()
        pvalues = model.pvalues
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = np.argmax(pvalues)
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.5}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


pct_test = 0.3

random_state = np.random.randint(0,1000)

keep_features = ['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip', 
                 'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']


for i in range(len(listVideo)):
        
    source = listVideo[i] #i

    X_src = pd.get_dummies(pd.DataFrame(np.array(source[keep_features], dtype =int)))
    
    y_src = np.array(source['elapsedtime'], dtype=float)
    X_src_train, X_src_test, y_src_train, y_src_test = train_test_split(X_src, 
                                                                        y_src, 
                                                                        test_size=pct_test, 
                                                                        random_state=random_state)
    
    """target = listVideo[j] #j

    X_tgt = target[['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
        'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']] 
    y_tgt = np.array(target['elapsedtime'], 
                     dtype=float)
    X_tgt_train, X_tgt_test, y_tgt_train, y_tgt_test = train_test_split(X_tgt, 
                                                                        y_tgt, 
                                                                        test_size=pct_test, 
                                                                        random_state=random_state)
    """
    
    result = stepwise_selection(X_src, y_src, verbose = False)

    #print('resulting features:')
    #print(result)

### (ii) Sampling

### (iii) Transfer

a = pd.DataFrame(np.array(source[['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
    'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']], dtype =int), columns=name)
a

name = ['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
    'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']


for i in range(len(name)):
    feature = a[name[i]]
    f_dummy = pd.get_dummies(feature, prefix = name[i], drop_first = True)
    col = f_dummy.columns
    for j in range(len(col)):
        print(f_dummy[col[j]])
    

a[8]

pd.get_dummies(source[['no_8x8dct','no_asm', 'no_cabac','no_deblock','no_fast_pskip',
    'no_mbtree','no_mixed_refs','no_weightb','rc_lookahead','ref']])